In [3]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.7 MB/s eta 0:00:00


In [8]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [16]:
import os
from dotenv import load_dotenv
from groq import Groq

# Load environment variables
load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# ==============================
# 1️⃣ Define Expert Configurations
# ==============================

MODEL_NAME = "llama-3.1-8b-instant"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Technical Support Expert.
Be precise, rigorous, and solution-focused.
Provide debugging steps, code fixes, and technical explanations.
If relevant, show corrected code snippets.
Avoid emotional language."""
    },
    "billing": {
        "system_prompt": """You are a Billing and Subscription Support Expert.
Be empathetic and professional.
Focus on refund policies, charges, invoices, and subscriptions.
Explain financial policies clearly and guide the user step-by-step."""
    },
    "general": {
        "system_prompt": """You are a friendly and helpful General Customer Support Assistant.
Handle casual conversations and general inquiries politely.
Provide clear and concise responses."""
    }
}


# ==============================
# 2️⃣ Router Function (MoE Gate)
# ==============================

def route_prompt(user_input: str) -> str:
    """
    Classifies the user input into:
    technical, billing, or general
    Returns ONLY the category name.
    """

    router_prompt = f"""
Classify the following user request into one of these categories:
[technical, billing, general]

Return ONLY the category name.

User request:
\"{user_input}\"
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0,  # deterministic routing
        messages=[
            {"role": "system", "content": "You are a strict classification engine."},
            {"role": "user", "content": router_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()

    # Safety fallback
    if category not in MODEL_CONFIG:
        category = "general"

    return category


# ==============================
# 3️⃣ Orchestrator
# ==============================

def process_request(user_input: str) -> str:
    """
    Full MoE pipeline:
    1. Route to expert
    2. Load expert system prompt
    3. Generate final response
    """

    category = route_prompt(user_input)

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=MODEL_NAME,
        temperature=0.7,  # creativity for experts
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content


# ==============================
# 4️⃣ Test Runner
# ==============================

if __name__ == "__main__":
    while True:
        user_query = input("\nEnter your query (or type 'exit'): ")

        if user_query.lower() == "exit":
            break

        result = process_request(user_query)
        print("\nResponse:\n", result)



Enter your query (or type 'exit'): how are you

Response:
 I'm doing great, thank you for asking. It's lovely to chat with you. How about you? Is there anything on your mind that you'd like to talk about or perhaps you have a question? I'm here to help.

Enter your query (or type 'exit'): I was charged twice for my subscription this month.

Response:
 I'm so sorry to hear that you've been charged twice for your subscription. I'm here to help you resolve this issue as quickly as possible.

To assist you further, could you please provide me with some information? 

1. Can you confirm the dates when you were charged for your subscription?
2. Are you using a credit card or another payment method?
3. Can you check your account and provide me with the exact amounts you were charged?
4. Are you experiencing any other issues with your subscription or account?

Once I have this information, I'll be happy to look into the matter and see what we can do to get you the refund you're owed.

Additio